# Sprint 4 Analysis

In [1]:
from arsenic import plotting, stats # useful FE analysis functions
import json
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import numpy as np
import seaborn as sns
from openeye import oechem
import numpy as np
from fah_xchem.analysis.constants import KT_KCALMOL # to convert from kT to kcal per mol

In [2]:
# Read in the analysis.json for Sprint 4
# Each transform is the backwards transform (?)
with open('../../data/sprint-4/analysis.json', 'r') as file:
    data = json.load(file)

In [4]:
data['series'].keys()

dict_keys(['metadata', 'compounds', 'transformations'])

In [9]:
data['series']['transformations'][0]['transformation']

{'run_id': 4,
 'xchem_fragment_id': 'x3110',
 'initial_microstate': {'compound_id': 'EN300-26619500',
  'microstate_id': 'EN300-26619500_2'},
 'final_microstate': {'compound_id': '3v3m-2020-04-Jacobs',
  'microstate_id': '3v3m-2020-04-Jacobs'}}

In [31]:
t['binding_free_energy']

{'point': 4.227757170041856, 'stderr': 0.5239797059936661}

In [34]:
s4_fe_store = {"start_compound": [], "start_compound_ms": [], "end_compound" : [], "DDG" : [], "dDDG": []}

for t in data['series']['transformations']:

    # if t['binding_free_energy']['point']< 0:

    # print(f"start compound: {t['transformation']['initial_microstate']['compound_id']}, start compound MS: {t['transformation']['initial_microstate']['microstate_id']}, final compound: {t['transformation']['final_microstate']['compound_id']}")
    # print(f"binding free energy: {t['binding_free_energy']['point']}")
    # print(' ')

    s4_fe_store["start_compound"].append(t['transformation']['initial_microstate']['compound_id'])
    s4_fe_store["start_compound_ms"].append(t['transformation']['initial_microstate']['microstate_id'])
    s4_fe_store["end_compound"].append(t['transformation']['final_microstate']['compound_id'])
    s4_fe_store["DDG"].append(t['binding_free_energy']['point'])
    s4_fe_store["dDDG"].append(t['binding_free_energy']['stderr'])

    # if not t['transformation']['final_microstate']['compound_id'] == "3v3m-2020-04-Jacobs":
    #     print("blah")
    

In [ ]:
s4_fe_store